In [ ]:
from imports import *

In [ ]:
# skriv årgang
aar = 2021

In [ ]:
# Hente data

training_data, imputatable_df, foretak_pub = ml_modeller.hente_training_data()

In [ ]:
def xgboost_model_mae(training_df, scaler, df_estimeres, GridSearch=True):
    """
    Trains an XGBoost model for predicting new_oms values with an optional GridSearch for hyperparameter tuning.

    Parameters:
    training_df (pd.DataFrame): DataFrame containing the training data.
    scaler (object): Scaler object for numerical features (e.g., StandardScaler, RobustScaler).
    df_estimeres (pd.DataFrame): DataFrame containing the data to be imputed.
    GridSearch (bool): Whether to perform GridSearch for hyperparameter tuning. Default is True.

    Returns:
    pd.DataFrame: DataFrame with predicted new_oms values.
    """
    import numpy as np
    import xgboost as xgb
    from sklearn.model_selection import train_test_split, GridSearchCV
    from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.compose import ColumnTransformer
    import matplotlib.pyplot as plt
    import shap

    # Make copies of the input DataFrames
    df = training_df.copy()
    imputed_df = df_estimeres.copy()
    
    # Drop rows with NaN values in the target column
    df = df.dropna(subset=['new_oms'])
    
    # Convert specified columns to category type
    categorical_columns = ["nacef_5", "tmp_sn2007_5", "b_kommunenr"]
    for col in categorical_columns:
        df[col] = df[col].astype("category")

    # Define features and target variable
    X = df.drop(columns=["new_oms"])
    y = df["new_oms"]

    # Define categorical and numerical features
    categorical_features = ["nacef_5", "tmp_sn2007_5", "b_kommunenr"]
    numerical_features = [
        "inntekt_delta_oms",
        "emp_delta_oms",
        "befolkning_delta_oms",
        "inflation_rate_oms",
        "gjeldende_bdr_syss",
        "new_oms_trendForecast", 
        'oms_syssmean_basedOn_naring',
        'oms_syssmean_basedOn_naring_kommune'
    ]

    # Preprocessing pipeline
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", scaler, numerical_features),  # Apply scaling to numerical features
            ("cat", OneHotEncoder(categories="auto", handle_unknown="ignore"), categorical_features),  # One-hot encoding for categorical features
        ]
    )

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Fit the preprocessor on the training data
    preprocessor.fit(X_train)

    # Transform the training and testing data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)

    if GridSearch:
        # Define the model
        regressor = xgb.XGBRegressor(eval_metric="rmse", random_state=42)

        # Define parameter grid for GridSearch
        param_grid = {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.2],
            'subsample': [0.8, 1.0],
            'colsample_bytree': [0.8, 1.0]
        }

        # Perform GridSearch with cross-validation
        grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3, verbose=1)
        grid_search.fit(X_train_transformed, y_train)

        # Print best parameters
        print("Best parameters found by GridSearch:", grid_search.best_params_)

        # Use best estimator from grid search
        regressor = grid_search.best_estimator_
    else:
        # Define the model with default parameters
        regressor = xgb.XGBRegressor(eval_metric="rmse", random_state=42)

        # Train the model
        eval_set = [(X_train_transformed, y_train), (X_test_transformed, y_test)]
        regressor.fit(X_train_transformed, y_train, eval_set=eval_set, verbose=False)

    # Evaluate the model
    y_pred = regressor.predict(X_test_transformed)

    # Check for negative values in predictions
    negative_indices = np.where(y_pred < 0)[0]
    negative_predictions = y_pred[y_pred < 0]

    #if len(negative_predictions) > 0:
    #    print("Number of negative predictions:", len(negative_predictions))
    #else:
    #    print("No negative predictions found.")
        
    # Set negative predictions to zero
    y_pred = np.maximum(y_pred, 0)

    # Calculate evaluation metrics
    mse = mean_squared_error(y_test, y_pred)
    r_squared = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)  # Calculate Mean Absolute Error
    #print("Mean Squared Error:", mse)
    #print("R-squared:", r_squared)
    #print("Mean Absolute Error:", mae)

    # Plot the learning history
    #results = regressor.evals_result()
    #epochs = len(results["validation_0"]["rmse"])
    #x_axis = range(0, epochs)
    #plt.figure(figsize=(10, 5))
    #plt.plot(x_axis, results["validation_0"]["rmse"], label="Train")
    #plt.plot(x_axis, results["validation_1"]["rmse"], label="Test")
    #plt.legend()
    #plt.xlabel("Epochs")
    #plt.ylabel("RMSE")
    #plt.title("XGBoost Learning History")
    #plt.show()

    # Plot Predicted vs. Actual Values
    #plt.figure(figsize=(10, 5))
    #plt.scatter(y_test, y_pred, alpha=0.3)
    #plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "r--", lw=2)
    #plt.xlabel("Actual")
    #plt.ylabel("Predicted")
    #plt.title("Predicted vs. Actual Values")
    #plt.show()

    # Plot Residuals
    #residuals = y_test - y_pred
    #plt.figure(figsize=(10, 5))
    #plt.scatter(y_test, residuals, alpha=0.3)
    #plt.hlines(0, y_test.min(), y_test.max(), colors="r", linestyles="dashed")
    #plt.xlabel("Actual")
    #plt.ylabel("Residuals")
    #plt.title("Residuals Plot")
    #plt.show()

    # Tree textual representation
    #booster = regressor.get_booster()
    #with open("dump.raw.txt", "w") as f:
    #    f.write("\n".join(booster.get_dump()))
    #print(booster.get_dump()[0])  # Print the first tree

    # SHAP values
    #explainer = shap.TreeExplainer(regressor, X_train_transformed)
    #shap_values = explainer.shap_values(X_test_transformed)

    # Get feature names after one-hot encoding
    #feature_names = preprocessor.get_feature_names_out()

    # Summary plot of SHAP values
    #shap.summary_plot(shap_values, X_test_transformed, feature_names=feature_names)

    # Force plot for a single prediction (e.g., the first instance)
    #shap.initjs()
    #shap.force_plot(explainer.expected_value, shap_values[0], X_test_transformed[0], feature_names=feature_names)

    # Find the correct index for the feature "verdi"
    #verdi_index = list(feature_names).index("num__new_oms_trendForecast")

    # Dependence plot to show the effect of a single feature across the dataset
    #shap.dependence_plot(verdi_index, shap_values, X_test_transformed, feature_names=feature_names)

    # Impute the missing data
    imputed_X = imputed_df.drop(columns=["new_oms"])
    imputed_X_transformed = preprocessor.transform(imputed_X)
    imputed_df["predicted_oms"] = regressor.predict(imputed_X_transformed)

    # Ensure no negative predictions
    imputed_df['predicted_oms'] = imputed_df['predicted_oms'].clip(lower=0)
    imputed_df['predicted_oms'] = imputed_df['predicted_oms'].astype(float)
    return mae, r_squared, negative_predictions


In [ ]:
# Choose Scaler (StandardScaler, MinMaxScaler, RobustScaler)
scaler = RobustScaler()

# Turn off GridSearch for faster run time
GridSearch=False

results = ml_modeller.xgboost_model(training_data, scaler, imputatable_df, GridSearch=GridSearch)

# Best result for GridSearch so far:

In [ ]:
# Choose Scaler (StandardScaler, MinMaxScaler, RobustScaler)
scaler = RobustScaler()

# Turn off GridSearch for faster run time
GridSearch=False

# Step 1: Extract first three digits from naring
training_data['naring_group'] = training_data['naring'].apply(lambda x: str(x)[:4])

# Create an empty list to store the results
results_list = []

# Step 2: Group by naring_group and iterate over each group
for naring_group, group_data in training_data.groupby('naring_group'):
    # Skip groups with fewer than 2 samples
    #print(naring_group, len(group_data))
    if len(group_data) < 2:
        print(f"Skipping group {naring_group} due to insufficient data.")
        continue
    
    # Step 3: Run the xgboost_model for each group
    mae, r_squared, negative_predictions = xgboost_model_mae(group_data, scaler, imputatable_df, GridSearch=GridSearch)
    
    
    # Store or print results (append to a list or DataFrame as needed)
    results_list.append([naring_group, mae,r_squared,len(negative_predictions), len(group_data)])

# Step 4: Create a DataFrame with the results
results_df = pd.DataFrame(results_list, columns=['naring_group', 'mae','r_squared', 'negative_predictions','observations in group'])

# Display the results table
#print(results_df)

In [ ]:
xgboost_model_mae_df=results_df

In [ ]:
results_df

In [ ]:
# Calculate the sum of the 'observations in group' column
total_observations = results_df['observations in group'].sum()

# Print the result
print("Total observations in group:", total_observations)


In [ ]:
def knn_model_mae(training_df, scaler, df_estimeres, GridSearch=True):
    """
    Trains a K-Nearest Neighbors model for predicting new_oms values with an optional GridSearch for hyperparameter tuning.

    Parameters:
    training_df (pd.DataFrame): DataFrame containing the training data.
    scaler (object): Scaler object for numerical features (e.g., StandardScaler, RobustScaler).
    df_estimeres (pd.DataFrame): DataFrame containing the data to be imputed.
    GridSearch (bool): Whether to perform GridSearch for hyperparameter tuning. Default is True.

    Returns:
    pd.DataFrame: DataFrame with predicted new_oms values.
    """
    import numpy as np
    from sklearn.model_selection import train_test_split, GridSearchCV
    from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.compose import ColumnTransformer
    from sklearn.neighbors import KNeighborsRegressor
    import matplotlib.pyplot as plt

    # Make copies of the input DataFrames
    df = training_df.copy()
    imputed_df = df_estimeres.copy()

    # Columns to fill with 'missing' and 0 respectively
    columns_to_fill = ["nacef_5", "tmp_sn2007_5", "b_kommunenr"]
    numeric_columns_to_fill = [
        "inntekt_delta_oms",
        "emp_delta_oms",
        "befolkning_delta_oms",
        "inflation_rate_oms",
        "gjeldende_bdr_syss",
        "new_oms_trendForecast",
        'oms_syssmean_basedOn_naring',
        'oms_syssmean_basedOn_naring_kommune'
    ]

    # Fill NaN values with 'missing' for the specified columns
    df[columns_to_fill] = df[columns_to_fill].fillna('missing')
    imputed_df[columns_to_fill] = imputed_df[columns_to_fill].fillna('missing')
    
    # Fill NaN values with 0 for the specified columns
    df[numeric_columns_to_fill] = df[numeric_columns_to_fill].fillna(0)
    imputed_df[numeric_columns_to_fill] = imputed_df[numeric_columns_to_fill].fillna(0)

    # Convert specified columns to category type
    categorical_columns = ["nacef_5", "tmp_sn2007_5", "b_kommunenr"]
    for col in categorical_columns:
        df[col] = df[col].astype("category")

    # Define features and target
    X = df.drop(columns=["new_oms"])
    y = df["new_oms"]

    # Define preprocessor
    categorical_features = ["nacef_5", "tmp_sn2007_5", "b_kommunenr"]
    numerical_features = [
        "inntekt_delta_oms",
        "emp_delta_oms",
        "befolkning_delta_oms",
        "inflation_rate_oms",
        "gjeldende_bdr_syss",
        "new_oms_trendForecast",
        'oms_syssmean_basedOn_naring',
        'oms_syssmean_basedOn_naring_kommune'
    ]

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", scaler, numerical_features),
            ("cat", OneHotEncoder(categories="auto", handle_unknown="ignore"), categorical_features),
        ]
    )

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Fit the preprocessor and transform the training and testing data
    preprocessor.fit(X_train)
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)

    if GridSearch:
        # Define the model
        regressor = KNeighborsRegressor()

        # Define parameter grid for GridSearch
        param_grid = {
            'n_neighbors': [2, 3, 5, 7]
        }

        # Perform GridSearch with cross-validation
        grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3, verbose=1)
        grid_search.fit(X_train_transformed, y_train)

        # Print best parameters
        print("Best parameters found by GridSearch:", grid_search.best_params_)

        # Use best estimator from grid search
        regressor = grid_search.best_estimator_
    else:
        # Define the model with default parameters
        regressor = KNeighborsRegressor(n_neighbors=2)

        # Train the model
        regressor.fit(X_train_transformed, y_train)

    # Predict on test data
    y_pred = regressor.predict(X_test_transformed)

    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    r_squared = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    #print("Mean Squared Error:", mse)
    #print("R-squared:", r_squared)
    #print("Mean Absolute Error:", mae)
    
    # Plot Predicted vs. Actual Values
    #plt.figure(figsize=(10, 5))
    #plt.scatter(y_test, y_pred, alpha=0.3)
    #plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "r--", lw=2)
    #plt.xlabel("Actual")
    #plt.ylabel("Predicted")
    #plt.title("Predicted vs. Actual Values")
    #plt.show()

    # Plot Residuals
    residuals = y_test - y_pred
    #plt.figure(figsize=(10, 5))
    #plt.scatter(y_test, residuals, alpha=0.3)
    #plt.hlines(0, y_test.min(), y_test.max(), colors="r", linestyles="dashed")
    #plt.xlabel("Actual")
    #plt.ylabel("Residuals")
    #plt.title("Residuals Plot")
    #plt.show()

    # Impute the missing data
    imputed_X = imputed_df.drop(columns=["new_oms"])
    imputed_X_transformed = preprocessor.transform(imputed_X)
    imputed_df["predicted_oms"] = regressor.predict(imputed_X_transformed)
    
    return mae, r_squared

In [ ]:
# Choose Scaler (StandardScaler, MinMaxScaler, RobustScaler)
scaler = RobustScaler()

# Turn off GridSearch for faster run time
GridSearch=False

# Step 1: Extract first three digits from naring
training_data['naring_group'] = training_data['naring'].apply(lambda x: str(x)[:4])

# Create an empty list to store the results
results_list = []

# Step 2: Group by naring_group and iterate over each group
for naring_group, group_data in training_data.groupby('naring_group'):
    # Skip groups with fewer than n_neighbors samples
    if len(group_data) < 3:  # Assuming n_neighbors = 2 for KNN
        print(f"Skipping group {naring_group} due to insufficient data.")
        continue
    
    # Step 3: Run the knn_model for each group
    mae, r_squared = knn_model_mae(group_data, scaler, imputatable_df, GridSearch=GridSearch)
    
    # Store or print results
    results_list.append([naring_group, mae, r_squared, len(group_data)])

# Step 4: Create a DataFrame with the results
results_df = pd.DataFrame(results_list, columns=['naring_group', 'mae', 'r_squared', 'observations in group'])

# Display the results table
#print(results_df)

In [ ]:
results_df

In [ ]:
knn_model_mae_df=results_df

In [ ]:


def nn_model_2_mae(training_df, scaler, epochs_number, batch_size, df_estimeres):
    
    import pandas as pd
    import numpy as np
    import tensorflow as tf
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.compose import ColumnTransformer
    import matplotlib.pyplot as plt
    from IPython.display import clear_output

    def build_nn_model(input_shape):
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Dense(64, input_shape=(input_shape,), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
        model.add(tf.keras.layers.Dropout(0.5))
        model.add(tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
        model.add(tf.keras.layers.Dropout(0.5))
        model.add(tf.keras.layers.Dense(1, activation='linear'))
        model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])
        return model

    df = training_df.copy()
    imputed_df = df_estimeres.copy()
 
    columns_to_fill = ["nacef_5", "tmp_sn2007_5", "b_kommunenr"]
    numeric_columns_to_fill = ["inntekt_delta_oms",
        "emp_delta_oms",
        "befolkning_delta_oms",
        "inflation_rate_oms",
        "gjeldende_bdr_syss",
        "new_oms_trendForecast",
        'oms_syssmean_basedOn_naring',
        'oms_syssmean_basedOn_naring_kommune']

    # Fill NaN values with 'missing' for the specified columns
    df[columns_to_fill] = df[columns_to_fill].fillna('missing')
    imputed_df[columns_to_fill] = imputed_df[columns_to_fill].fillna('missing')
    
    df[numeric_columns_to_fill] = df[numeric_columns_to_fill].fillna(0)
    imputed_df[numeric_columns_to_fill] = imputed_df[numeric_columns_to_fill].fillna(0)

    categorical_columns = ["nacef_5", "tmp_sn2007_5", "b_kommunenr"]
    for col in categorical_columns:
        df[col] = df[col].astype("category")

    categorical_features = categorical_columns

    numerical_features = [
        "inntekt_delta_oms",
        "emp_delta_oms",
        "befolkning_delta_oms",
        "inflation_rate_oms",
        "gjeldende_bdr_syss",
        "new_oms_trendForecast",
        'oms_syssmean_basedOn_naring',
        'oms_syssmean_basedOn_naring_kommune'
    ]

    X = df.drop(columns=["new_oms"])
    y = df["new_oms"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", scaler, numerical_features),
            ("cat", OneHotEncoder(categories="auto", handle_unknown="ignore"), categorical_features),
        ]
    )

    X_train_transformed = preprocessor.fit_transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    input_shape = X_train_transformed.shape[1]

    model = build_nn_model(input_shape)
    
    # Define learning rate scheduler callback
    def scheduler(epoch, lr):
        if epoch < 60:
            return lr
        else:
            return float(lr * tf.math.exp(-0.1))
    
    lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Real-time plotting function
    def live_plotting(epoch, logs):
        loss.append(logs['loss'])
        val_loss.append(logs['val_loss'])
        epochs.append(epoch)
        
        clear_output(wait=True)  # Clears the output of the cell
        plt.figure(figsize=(10, 5))
        plt.plot(epochs, loss, 'b', label='Training loss')
        plt.plot(epochs, val_loss, 'r', label='Validation loss')
        plt.title('Training and Validation Loss')
        plt.legend()
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.show()

    # Lists to hold the values for plotting
    loss, val_loss, epochs = [], [], []
    #plot_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=live_plotting)
    
    history = model.fit(
        X_train_transformed, y_train,
        validation_split=0.2,
        epochs=epochs_number,
        batch_size=batch_size,
        #callbacks=[early_stopping, lr_scheduler, plot_callback],  # Add the plotting callback
        verbose=0
    )

    y_pred = model.predict(X_test_transformed).flatten()  # Ensure y_pred is 1-dimensional

    # Set negative predictions to zero
    y_pred = np.maximum(y_pred, 0)

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    medae = median_absolute_error(y_test, y_pred)
    r_squared = r2_score(y_test, y_pred)
    rmse = np.sqrt(mse)
    mean_y_test = np.mean(y_test)

    #print(f"Mean Squared Error (MSE): {mse}")
    #print(f"Mean Absolute Error (MAE): {mae}")
    #print(f"Median Absolute Error (MedAE): {medae}")
    #print(f"R-squared score: {r_squared}")

    #plt.figure(figsize=(10, 5))
    #plt.scatter(y_test, y_pred, alpha=0.3)
    #plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "r--", lw=2)
    #plt.xlabel("Actual")
    #plt.ylabel("Predicted")
    #plt.title("Predicted vs. Actual Values")
    #plt.show()

    residuals = y_test - y_pred
    #plt.figure(figsize=(10, 5))
    #plt.scatter(y_test, residuals, alpha=0.3)
    #plt.hlines(0, y_test.min(), y_test.max(), colors="r", linestyles="dashed")
    #plt.xlabel("Actual")
    #plt.ylabel("Residuals")
    #plt.title("Residuals Plot")
    #plt.show()

    # Predict values for the imputed_df
    imputed_X = imputed_df.drop(columns=["new_oms"])
    imputed_X_transformed = preprocessor.transform(imputed_X)
    imputed_df["predicted_oms"] = model.predict(imputed_X_transformed)

    return mae, r_squared


In [ ]:
# choose Scaler (StandardScaler, MinMaxScaler, RobustScaler)
scaler = RobustScaler()

# choose epoch number and batch size (more = faster, less = possibly better learning/convergence ) If using GridSearch these numbers wont matter.
# GridSearch will take a long time to run. 
epochs_number = 200
batch_size = 500
GridSearch=False

# results = ml_modeller.nn_model_1(training_data, scaler, epochs_number, batch_size, imputatable_df, GridSearch=GridSearch)
#results = ml_modeller.nn_model_2(training_data, scaler, epochs_number, batch_size, imputatable_df)

# Step 1: Extract first three digits from naring
training_data['naring_group'] = training_data['naring'].apply(lambda x: str(x)[:4])

# Create an empty list to store the results
results_list = []

# Step 2: Group by naring_group and iterate over each group
for naring_group, group_data in training_data.groupby('naring_group'):
    # Skip groups with fewer than n_neighbors samples
    if len(group_data) < 3:  # Assuming n_neighbors = 2 for KNN
        print(f"Skipping group {naring_group} due to insufficient data.")
        continue
    
    # Step 3: Run the knn_model for each group
    mae, r_squared = nn_model_2_mae(group_data, scaler, epochs_number, batch_size, imputatable_df)
    
    # Store or print results
    results_list.append([naring_group, mae, r_squared, len(group_data)])

# Step 4: Create a DataFrame with the results
results_df = pd.DataFrame(results_list, columns=['naring_group', 'mae', 'r_squared', 'observations in group'])

# Display the results table
#print(results_df)

In [ ]:
nn_model_2_mae=results_df

In [ ]:
nn_model_2_mae

In [ ]:
def hente_training_data_mae():
    
    fil_path = [
        f
        for f in fs.glob(
            f"gs://ssb-prod-noeku-data-produkt/temp/training_data.parquet"
        )
        if f.endswith(".parquet")
    ]

    # Use the ParquetDataset to read multiple files
    dataset = pq.ParquetDataset(fil_path, filesystem=fs)
    table = dataset.read()

    # Convert to Pandas DataFrame
    training_data = table.to_pandas()
        
    fil_path = [
        f
        for f in fs.glob(
            f"gs://ssb-prod-noeku-data-produkt/temp/imputatable_df.parquet"
        )
        if f.endswith(".parquet")
    ]

    # Use the ParquetDataset to read multiple files
    dataset = pq.ParquetDataset(fil_path, filesystem=fs)
    table = dataset.read()

    # Convert to Pandas DataFrame
    imputatable_df = table.to_pandas()
    
    fil_path = [
        f
        for f in fs.glob(
            f"gs://ssb-prod-noeku-data-produkt/statistikkfiler/g2021/statistikkfil_foretak_pub.parquet"
        )
        if f.endswith(".parquet")
    ]

    # Use the ParquetDataset to read multiple files
    dataset = pq.ParquetDataset(fil_path, filesystem=fs)
    table = dataset.read()

    # Convert to Pandas DataFrame
    foretak_pub = table.to_pandas()
    
    foretak_pub['n3'] = foretak_pub['naring_f'].str[:4]
    foretak_pub['n2'] = foretak_pub['naring_f'].str[:2]
    
    #foretak_pub = foretak_pub[foretak_pub['n2'].isin(['45', '46', '47'])]
    foretak_pub = foretak_pub[['n3',
                             'bearbeidingsverdi',
                             'produktinnsats',
                             'produksjonsverdi',
                             'omsetning',
                             'sysselsetting_syss',
                             'ts_forbruk',
                            'ts_avanse',
                            'ts_salgsint',
                            'ts_vikarutgifter',
                            'ts_byggvirk',
                            'ts_varehan',
                            'ts_anlegg',
                            'ts_tjeneste',
                            'ts_industri',
                            'ts_agentur',
                            'ts_detalj',
                            'ts_engros',
                            'ts_internet_salg',
                            'ts_annet',
                            'nopost_lonnskostnader',
                            'nopost_driftskostnader',
                            'nopost_driftsresultat',
                            'nopost_driftsinntekter',
                            'saldo_kjop_p0580']]
        
    return training_data, imputatable_df, foretak_pub

In [ ]:
training_data, imputatable_df, foretak_pub = hente_training_data_mae()

In [ ]:
def xgboost_n3_klass_mae(df):
    """
    Trains an XGBoost classifier to predict 'n3' categories with preprocessing for numerical and categorical data.

    Parameters:
    df (pd.DataFrame): DataFrame containing the data with 'n3' as the target variable.

    Returns:
    None
    """
    import pandas as pd
    import numpy as np
    import xgboost as xgb
    from sklearn.model_selection import train_test_split, learning_curve
    from sklearn.metrics import accuracy_score, classification_report
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.compose import ColumnTransformer
    from sklearn.pipeline import Pipeline
    from sklearn.impute import SimpleImputer
    import matplotlib.pyplot as plt

    # Filter out sparse classes
    min_samples_per_class = 50
    value_counts = df['n3'].value_counts()
    to_remove = value_counts[value_counts < min_samples_per_class].index
    df = df[~df['n3'].isin(to_remove)]

    # Convert target to integer labels and store the mapping
    labels, unique = pd.factorize(df['n3'])
    df['n3_encoded'] = labels
    n3_mapping = dict(zip(labels, unique))

    # Identify categorical and numerical columns excluding the target
    non_feature_cols = ['n3']  # 'n3' is now not a feature
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    categorical_cols = [col for col in categorical_cols if col not in non_feature_cols]
    numerical_cols = df.select_dtypes(include=['number']).columns.tolist()
    numerical_cols.remove('n3_encoded')  # Assume 'n3_encoded' is the new target variable

    # Preprocessing for numerical data: simple imputer with median strategy
    numerical_transformer = SimpleImputer(strategy='median')

    # Preprocessing for categorical data: impute missing values and apply one-hot encoding
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Bundle preprocessing for numerical and categorical data
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ])

    # Define the XGBoost classifier
    model = xgb.XGBClassifier(objective='multi:softprob', random_state=42, eval_metric='mlogloss')

    # Create a pipeline
    clf = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

    # Prepare data
    X = df.drop(non_feature_cols + ['n3_encoded'], axis=1)
    y = df['n3_encoded']  # Correct target variable

    # Split the data with stratification
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Fit the model
    clf.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = clf.predict(X_test)

    # Generate a mapping from labels to original 'n3' categories if not done previously
    n3_mapping = {idx: label for idx, label in enumerate(pd.unique(df['n3']))}

    # Safely convert predictions and true values back to original labels using the mapping
    y_pred_labels = [n3_mapping.get(label, 'Unknown') for label in y_pred]
    y_test_labels = [n3_mapping.get(label, 'Unknown') for label in y_test]

    # Print accuracy and classification report using the original 'n3' labels
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test_labels, y_pred_labels))
    
    mae = mean_absolute_error(y_test, y_pred)
    #medae = median_absolute_error(y_test, y_pred)
    r_squared = r2_score(y_test, y_pred)
    
    # Plot learning curve
    #def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None, n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    #    """
    #    Generate a simple plot of the test and training learning curve.

    #    Parameters:
    #    estimator: object type that implements the "fit" and "predict" methods
    #    title: string, Title for the chart
    #    X: array-like, shape (n_samples, n_features), Training vector
    #    y: array-like, shape (n_samples) or (n_samples, n_features), Target relative to X for classification or regression
    #    ylim: tuple, shape (ymin, ymax), optional, Defines minimum and maximum y-values plotted
    #    cv: int, cross-validation generator or an iterable, optional, Determines the cross-validation splitting strategy
    #    n_jobs: int or None, optional, Number of jobs to run in parallel
    #    train_sizes: array-like, shape (n_ticks,), Relative or absolute numbers of training examples that will be used to generate the learning curve

    #    Returns:
    #    plt: Matplotlib plot object
    #    """
    #    plt.figure()
    #    plt.title(title)
    #    if ylim is not None:
    #        plt.ylim(*ylim)
    #    plt.xlabel("Training examples")
    #    plt.ylabel("Score")
    #    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    #    train_scores_mean = np.mean(train_scores, axis=1)
    #    train_scores_std = np.std(train_scores, axis=1)
    #    test_scores_mean = np.mean(test_scores, axis=1)
    #    test_scores_std = np.std(test_scores, axis=1)
    #    plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="r")
    #    plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color="g")
    #    plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
    #    plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")
    #    plt.legend(loc="best")
    #    return plt

    #plot_learning_curve(clf, "Learning Curve for XGBoost Classifier", X_train, y_train, cv=5)
    #plt.show()
    print(mae)
    return mae, r_squared


In [ ]:
results = xgboost_n3_klass_mae(foretak_pub)

In [ ]:
results

In [ ]:
# choose Scaler (StandardScaler, MinMaxScaler, RobustScaler)
scaler = RobustScaler()

# choose epoch number and batch size (more = faster, less = possibly better learning/convergence ) If using GridSearch these numbers wont matter.
# GridSearch will take a long time to run. 

GridSearch=False

# results = ml_modeller.nn_model_1(training_data, scaler, epochs_number, batch_size, imputatable_df, GridSearch=GridSearch)
#results = ml_modeller.nn_model_2(training_data, scaler, epochs_number, batch_size, imputatable_df)

# Step 1: Extract first three digits from naring
#foretak_pub['n3'] = foretak_pub['n3'].apply(lambda x: str(x)[:4])

# Create an empty list to store the results
# Create an empty list to store the results
results_list = []

# Step 2: Group by 'n3' and iterate over each group
for n3, group_data in foretak_pub.groupby('n3'):
    # Skip groups with fewer than n_neighbors samples (in this case, fewer than 3 samples)
    if len(group_data) < 3:
        print(f"Skipping group {n3} due to insufficient data.")
        continue

    # Check if 'n3_encoded' exists or create it by factorizing the 'n3' column
    if 'n3_encoded' not in group_data.columns:
        labels, unique = pd.factorize(group_data['n3'])
        group_data['n3_encoded'] = labels
    
    # Ensure the group has enough samples for train/test split
    if len(group_data) < 5:  # 20% test size needs at least 5 samples
        print(f"Skipping group {n3} due to insufficient data for train/test split.")
        continue

    # Step 3: Run the xgboost_n3_klass_mae model for each group
    try:
        mae, r_squared = xgboost_n3_klass_mae(group_data)
    except ValueError as e:
        print(f"Error processing group {n3}: {e}")
        continue
    
    # Store results (n3 group, mae, r_squared, and number of observations in that group)
    results_list.append([n3, mae, r_squared, len(group_data)])

# Step 4: Create a DataFrame with the results
results_df = pd.DataFrame(results_list, columns=['n3', 'mae', 'r_squared', 'observations'])



# Display the results table
#print(results_df)

In [ ]:
results_df

In [ ]:
results = ml_modeller.knn_n3_klass(foretak_pub)

In [ ]:
def knn_n3_klass_mae(df):
    """
    Trains a K-Nearest Neighbors (KNN) classifier to predict 'n3' categories with preprocessing 
    for numerical and categorical data.

    Parameters:
    df (pd.DataFrame): DataFrame containing the data with 'n3' as the target variable.

    Returns:
    None
    """
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split, learning_curve
    from sklearn.metrics import accuracy_score, classification_report
    from sklearn.preprocessing import OneHotEncoder, StandardScaler
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.compose import ColumnTransformer
    from sklearn.pipeline import Pipeline
    from sklearn.impute import SimpleImputer
    import matplotlib.pyplot as plt

    # Filter out sparse classes
    min_samples_per_class = 100
    value_counts = df['n3'].value_counts()
    to_remove = value_counts[value_counts < min_samples_per_class].index
    df = df[~df['n3'].isin(to_remove)]

    # Identify categorical and numerical columns
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    numerical_cols = df.select_dtypes(include=['number']).columns.tolist()

    # Specify columns to exclude from features
    non_feature_cols = ['n3']
    categorical_cols = [col for col in categorical_cols if col not in non_feature_cols]

    # Preprocessing for numerical data: impute missing values and scale
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())  # Scale features
    ])

    # Preprocessing for categorical data: impute missing values and apply one-hot encoding
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Bundle preprocessing for numerical and categorical data
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ])

    # Define the KNN classifier
    model = KNeighborsClassifier(n_neighbors=5)

    # Create a pipeline
    clf = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

    # Prepare data
    X = df.drop(non_feature_cols, axis=1)
    y = df['n3']

    # Split the data with stratification
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Fit the model
    clf.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = clf.predict(X_test)
    #print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
    #print(classification_report(y_test, y_pred))

    # Plot learning curve
    """
    #def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None, n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
        #
        Generate a simple plot of the test and training learning curve.

        Parameters:
        estimator: object type that implements the "fit" and "predict" methods
        title: string, Title for the chart
        X: array-like, shape (n_samples, n_features), Training vector
        y: array-like, shape (n_samples) or (n_samples, n_features), Target relative to X for classification or regression
        ylim: tuple, shape (ymin, ymax), optional, Defines minimum and maximum y-values plotted
        cv: int, cross-validation generator or an iterable, optional, Determines the cross-validation splitting strategy
        n_jobs: int or None, optional, Number of jobs to run in parallel
        train_sizes: array-like, shape (n_ticks,), Relative or absolute numbers of training examples that will be used to generate the learning curve

        Returns:
        plt: Matplotlib plot object
        #
        plt.figure()
        plt.title(title)
        if ylim is not None:
            plt.ylim(*ylim)
        plt.xlabel("Training examples")
        plt.ylabel("Score")
        train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
        train_scores_mean = np.mean(train_scores, axis=1)
        train_scores_std = np.std(train_scores, axis=1)
        test_scores_mean = np.mean(test_scores, axis=1)
        test_scores_std = np.std(test_scores, axis=1)
        plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="r")
        plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color="g")
        plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
        plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")
        plt.legend(loc="best")
        return plt
    """
    mae = mean_absolute_error(y_test, y_pred)
    #medae = median_absolute_error(y_test, y_pred)
    r_squared = r2_score(y_test, y_pred)
    return mae , r_squared

    #plot_learning_curve(clf, "Learning Curve for KNN Classifier", X_train, y_train, cv=5)
    #plt.show()


In [ ]:
results = knn_n3_klass_mae(foretak_pub)

In [ ]:
results

In [ ]:
scaler = RobustScaler()

# choose epoch number and batch size (more = faster, less = possibly better learning/convergence ) If using GridSearch these numbers wont matter.
# GridSearch will take a long time to run. 

GridSearch=False

# results = ml_modeller.nn_model_1(training_data, scaler, epochs_number, batch_size, imputatable_df, GridSearch=GridSearch)
#results = ml_modeller.nn_model_2(training_data, scaler, epochs_number, batch_size, imputatable_df)

# Step 1: Extract first three digits from naring
#foretak_pub['n3'] = foretak_pub['n3'].apply(lambda x: str(x)[:4])

# Create an empty list to store the results
# Create an empty list to store the results
results_list = []

# Step 2: Group by 'n3' and iterate over each group
for n3, group_data in foretak_pub.groupby('n3'):
    # Skip groups with fewer than n_neighbors samples (in this case, fewer than 3 samples)
    if len(group_data) < 3:
        print(f"Skipping group {n3} due to insufficient data.")
        continue

    # Check if 'n3_encoded' exists or create it by factorizing the 'n3' column
    if 'n3_encoded' not in group_data.columns:
        labels, unique = pd.factorize(group_data['n3'])
        group_data['n3_encoded'] = labels
    
    # Ensure the group has enough samples for train/test split
    if len(group_data) < 5:  # 20% test size needs at least 5 samples
        print(f"Skipping group {n3} due to insufficient data for train/test split.")
        continue

    # Step 3: Run the xgboost_n3_klass_mae model for each group
    try:
        mae, r_squared = knn_n3_klass_mae(group_data)
    except ValueError as e:
        print(f"Error processing group {n3}: {e}")
        continue
    
    # Store results (n3 group, mae, r_squared, and number of observations in that group)
    results_list.append([n3, mae, r_squared, len(group_data)])

# Step 4: Create a DataFrame with the results
results_df = pd.DataFrame(results_list, columns=['n3', 'mae', 'r_squared', 'observations'])



# Display the results table
#print(results_df)

In [ ]:
results_df

In [ ]:
def lstm_model_mae(training_df, scaler, df_estimeres, time_steps=60, epochs=20, batch_size=32):
    """
    Trains an LSTM model for predicting new_oms values.

    Parameters:
    training_df (pd.DataFrame): DataFrame containing the training data.
    scaler (object): Scaler object for numerical features (e.g., StandardScaler, RobustScaler).
    df_estimeres (pd.DataFrame): DataFrame containing the data to be imputed.
    time_steps (int): Number of previous time steps used to predict the next step.
    epochs (int): Number of epochs for model training.
    batch_size (int): Batch size for model training.

    Returns:
    float: Mean Absolute Error (MAE) and R-squared score.
    """
    import numpy as np
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
    from sklearn.compose import ColumnTransformer
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout
    from sklearn.preprocessing import OneHotEncoder

    # Make copies of the input DataFrames
    df = training_df.copy()
    imputed_df = df_estimeres.copy()
    
    # Drop rows with NaN values in the target column
    df = df.dropna(subset=['new_oms'])
    
    # Convert specified columns to category type
    categorical_columns = ["nacef_5", "tmp_sn2007_5", "b_kommunenr"]
    for col in categorical_columns:
        df[col] = df[col].astype("category")

    # Define features and target variable
    X = df.drop(columns=["new_oms"])
    y = df["new_oms"]

    # Define categorical and numerical features
    categorical_features = ["nacef_5", "tmp_sn2007_5", "b_kommunenr"]
    numerical_features = [
        "inntekt_delta_oms",
        "emp_delta_oms",
        "befolkning_delta_oms",
        "inflation_rate_oms",
        "gjeldende_bdr_syss",
        "new_oms_trendForecast", 
        'oms_syssmean_basedOn_naring',
        'oms_syssmean_basedOn_naring_kommune'
    ]

    # Preprocessing pipeline
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", scaler, numerical_features),  # Apply scaling to numerical features
            ("cat", OneHotEncoder(categories="auto", handle_unknown="ignore"), categorical_features),  # One-hot encoding for categorical features
        ]
    )

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Fit the preprocessor on the training data
    preprocessor.fit(X_train)

    # Transform the training and testing data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)

    # Reshape the data into time steps for LSTM
    def create_time_series_data(X, y, time_steps):
        Xs, ys = [], []
        for i in range(time_steps, len(X)):
            Xs.append(X[i - time_steps:i])
            ys.append(y[i])
        return np.array(Xs), np.array(ys)

    # Reshape training and test data to 3D for LSTM
    X_train_series, y_train_series = create_time_series_data(X_train_transformed, y_train, time_steps)
    X_test_series, y_test_series = create_time_series_data(X_test_transformed, y_test, time_steps)

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_series.shape[1], X_train_series.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the LSTM model
    model.fit(X_train_series, y_train_series, epochs=epochs, batch_size=batch_size, validation_data=(X_test_series, y_test_series))

    # Make predictions
    y_pred_series = model.predict(X_test_series)

    # Calculate evaluation metrics
    y_pred_series = np.maximum(y_pred_series, 0)  # Ensure no negative predictions
    mse = mean_squared_error(y_test_series, y_pred_series)
    r_squared = r2_score(y_test_series, y_pred_series)
    mae = mean_absolute_error(y_test_series, y_pred_series)

    # Impute the missing data
    imputed_X = imputed_df.drop(columns=["new_oms"])
    imputed_X_transformed = preprocessor.transform(imputed_X)

    # Reshape imputed data for prediction
    imputed_X_series = np.array([imputed_X_transformed[i - time_steps:i] for i in range(time_steps, len(imputed_X_transformed))])
    imputed_df["predicted_oms"] = model.predict(imputed_X_series)

    # Ensure no negative predictions
    imputed_df['predicted_oms'] = imputed_df['predicted_oms'].clip(lower=0)
    imputed_df['predicted_oms'] = imputed_df['predicted_oms'].astype(float)

    return mae, r_squared
